# Analysing output of t2p
This is a short script we will use as a demo on how to use the outputs of t2p for longitudinal analysis, but for now it is a practice notebook for Manon :)

The example here is for a 1 plane recording with simultaneous videography (given dataset is jm032). Track2p should be run on days (including) 19.10.2023 - 23.10.2023.

### Task description:

We want to know:
- Are there neurons in our recordings that are correlated with movement
- If yes, how stable is this correlation (are the same neurons correlated with movement on different days?, this can only be done with matched cells)
- Is this the case for all ages? If not when does this correlation arise?

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import os

### Step by step guide (more detailed explanations below):

Each point from this list matches one section of this notebook

1) Load the output of track2p
2) Find cells that are present in all recordings ('matched cells')
3) 
4) Load the activity of the matched cells and visualise it
5) Load the 'movement' data 
6) Study the correlation of neural and movement data

### 1) Load the output of track2p
The simplest way to do this is to load the `.npy` file: `t2p_output_path/track2p/plane#_match_mat.npy`. In this example it is simple because we only have one plane.

In [ ]:
# first load the t2p ops
t2p_save_path = # ... add your code ...

In [ ]:
# use np.load()
t2p_match_mat = # ... add your code ...

In [ ]:
# what is the shape of this matrix?
# what do you think the rows and columns represent?
print(t2p_match_mat.shape)

### 2) Find cells that are present in all recordings ('matched cells')



Now from this matrix get the matches that are present on all days (Hint: use the information below from the README)

- A matrix (`plane#_match_mat.npy`) containing the indices of matched neurons across the session for a given plane (`#` is the index of the plane). Since matching is done from first day to last, some neurons will not be sucessfully tracked after one or a few days. In this case the matrix contains `None` values. To get neurons tracked across all days only take the rows of the matrices containing no `None` values. 

In [ ]:
# get the rows that do not contain any Nones
t2p_match_mat_allday = # ... add your code ...

In [ ]:
# what is the shape of this new matrix? (this is the number of cells that were successfully matched across all days)
print(t2p_match_mat_allday.shape)

### 3) Load the data from one example dataset and visualise it


Hint: A nice way to implement this is to take the data saved in track_ops.npy file (for example to access the paths of the datasets used for t2p, see example below)

In [ ]:
track_ops = #... add your code ...

In [ ]:
# to show which datasets were used when t2p was run
print('Datasets used for t2p:\n')
for ds_path in track_ops.all_ds_path:
    print(ds_path)

For this part it is important to know a bit about how the suite2p structures the outputs: https://suite2p.readthedocs.io/en/latest/outputs.html (the important things will be the `ops.npy`, `stat.npy`, `iscell.npy` and the `F.npy`)
As an exercise you can first do it for a single dataset.

In [ ]:
# lets take the last dataset
last_ds_path = track_ops.all_ds_path[-1]
print(f'We will look at the dataset saved at: {last_ds_path}')

In [ ]:
# load the three files
last_ops = # ... add your code ...
last_stat = # ... add your code ...
last_f = # ... add your code ...
iscell = # ... add your code ...

In [ ]:
# now first plot the mean image of the movie (it is saved in ops.npy, for more infor see the suite2p outputs documentation)
mean_img = # ... add your code ...


In [ ]:
# next scatter the location of the first n cells onto the average image (use the stat.npy file, and plot the median coordinates for each cell) start with first 10 cells, then try more (you can also try to show all)
# Hint: Be careful about getting the x and y coordinates correct (images use a different convention than sccatter), you can verify you did it correctly if the scatter points are on top of the cells
n = 10

# plot mean image as above
# ... add your code ...


#scatter the first n cells
# ... add your code ...


In [ ]:
# if you plot all entries from stat you can see that there are a lot of things detected (too many to be cells), so we need to filter them out
# luckily suite2p tells us how confident it is that something is a cell in the iscell.npy file (see the suite2p outputs documentation for more info)

# lets use the same confidence that we used when we ran track2p
iscell_thr = track_ops.iscell_thr
print(f'The iscell threshold used was: {iscell_thr}')

In [ ]:
# now use the iscell.npy file to filter out the cells that are not confident enough (use the second column of the iscell.npy file)
stat_iscell = # ... add your code ...

In [ ]:
print(f'Got {stat_iscell.shape[0]} (/{last_stat.shape[0]}) cells using the iscell threshold')

In [ ]:
# now plot the median coordinates of the cells that are confident enough
# Hint: you can use the same code as above, but use the new stat_iscell instead of stat and loop through all elements
n = stat_iscell.shape[0]

# ... add your code ...


In [ ]:
# good! now we can move on to looking at the activity of the cells. First plot the activity of the first cell as a time-series (use the F.npy file, the cells here are matched to the cells in stat.npy)
print(f'len of stat: {len(last_stat)}')
print(f'shape of F: {last_f.shape}')


In [ ]:
# you can see that the rows of F are the cells and the columns are the frames of the movie
# lets plot the first cell (hint: use a wide figure)
plt.figure(figsize=(10, 1))
i = 0

# plot the time series below

# ... add your code ...


Now lets plot the activity of all cells as a 'raster plot'.
A raster plot is just a heatmap of the F matrix. By convention we use black as the cell being activated (high value) and white as the cell not being activated (low value).
For visualisation purposes we also normalize each row by z-scoring (you can try with or without to see the difference). 

Hint: Set the vmin to 0 and vmax to 1.96 in the imshow to see the activity better.

For example of how a raster should look, see figure G or H [here](https://www.science.org/cms/10.1126/science.aav7893/asset/1f184dca-7c86-432c-8d81-b4529619480d/assets/graphic/364_aav7893_f1.jpeg).

In [ ]:
from scipy.stats import zscore

In [ ]:
# also filter by the iscell threshold
last_f_iscell = # ... add your code ...

In [ ]:
# (hint: use the imported zscore function from scipy.stats) being careful which axis you zscore over
last_f_iscell_zscore = # ... add your code ...

In [ ]:
# check that row 0 is now zscored (mean should be 0 and std should be 1)
row_std = np.std(last_f_iscell_zscore[0, :])
row_mean = np.mean(last_f_iscell_zscore[0, :])
print(f'row 0 std: {row_std}')
print(f'row 0 mean: {row_mean}')

In [ ]:
# now plot the raster using last_f_iscell_zscore (make sure to follow instructions above, colormap, vmin, vmax, etc.)

# ... add your code ...


Extra task: People often plot the mean activity of all cells for each timepoint as a subplot below a raster. Implement this as an extra task.

In [ ]:

# ... add your code ...


## 4) Load the activity of the matched cells and visualise it

Now that we know how to look at data in one recording we will use the optput from track2p to look at activity of the same cells across all datasets.